In [ ]:
import pandas as pd
import clipboard
from sqlalchemy import create_engine
import regex as re
import time
from datetime import datetime
import os
import glob

In [ ]:
all_data = pd.DataFrame()
allExcelFiles =glob.glob("/home/trishalaswain/Downloads/SCP_MAY_SHEETS/*31.tsv")
scp_july_combined = pd.concat(pd.read_table(excelFile, index_col=False) for excelFile in allExcelFiles)

In [ ]:
scp_july_combined.columns

In [ ]:
scp_july_combined = scp_july_combined[['id', 'accountNumber', 'EANDM_Suggested', 'EANDM_Final']]

In [ ]:
scp_july_combined.to_excel('/home/trishalaswain/ClientData/SCP/EPIC_SCP/MAY/SCP_MAY_CODES.xlsx', index=False)

In [ ]:
df = pd.read_csv('/home/trishalaswain/Documents/machine_learning/pred_july.csv', sep='\t', index_col=False)

In [ ]:
df.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1, inplace=True)

In [ ]:
df

In [ ]:
def highlight_same(data):
    
    highlighter = pd.DataFrame("", index=data.index, columns=data.columns)
    for col in data.columns[:-1]:
        for index in data.index[:-1]:
            # if 'Total' in index or 'Total' in col:
            #     highlighter.loc[index, col] = 'background-color : yellow'
 

            if col == index:
                highlighter.loc[index, col] = 'background-color : springgreen'
                
            elif str(col)>str(index) and data.loc[index, col]>100:
                highlighter.loc[index, col] = 'background-color : lightskyblue'

            elif data.loc[index, col]>100:
                highlighter.loc[index, col] = 'background-color : PaleVioletRed'

    return highlighter
# crosstab = cs.style.apply(highlight_same, axis=None)

In [ ]:
pd.crosstab(df['EANDM_Final_code_1'], df['ypred_july'], margins=True, margins_name='Total').style.apply(highlight_same, axis=None)

In [ ]:
print(df[(df.EANDM_Final_code_1 == 2) & (df.ypred_july == 3)].account_number.to_list())

In [ ]:
# print(df[(df.EANDM_Final_code_1 == 2) & (df.ypred_july == 3)].head(30).MDM_SENTS.to_markdown())

In [ ]:
df_mismatch = df[(df.EANDM_Final_code_1 == 2) & (df.ypred_july == 3)]

In [ ]:
# df_mismatch[df_mismatch['MDM_SENTS'].lower().contains('ems ')]
# df['EMS_CASES'] = 0
# for i, r in df.iterrows():
#     if ' ems ' in r['MDM_SENTS']:
#         df.loc[i, 'EMS_CASES'] = 1

In [ ]:
# df[df.EMS_CASES == 1].EANDM_Final_code.value_counts()

In [ ]:
# import scipy.sparse
# import pickle
# vectorizer_filename = "/home/trishalaswain/Documents/machine_learning/tfidf.pkl"
# model_filename1 = "/home/trishalaswain/Documents/machine_learning/LR.pkl"

# model=pickle.load(open(model_filename1, "rb"))
# vectorizer =pickle.load(open(vectorizer_filename, "rb"))

In [ ]:
df_mismatch.columns

In [ ]:
data = df_mismatch.head(20).clean_sen_MDM_SENTS.to_list()

In [ ]:
import pickle
import eli5
from sklearn.feature_extraction.text import TfidfVectorizer

# Load the trained logistic regression model
with open('/home/trishalaswain/Documents/machine_learning/LR.pkl', 'rb') as model_file:
    model = pickle.load(model_file)

# Load the vectorizer
with open('/home/trishalaswain/Documents/machine_learning/tfidf.pkl', 'rb') as vectorizer_file:
    vectorizer = pickle.load(vectorizer_file)

# Define some example data
# data = ["This is an example sentence.",
#         "Another example for interpretation."]

# Transform the data using the vectorizer
X = vectorizer.transform(data)

# Predict the model's output for the given data
predictions = model.predict(X)

# Get the feature names from the vectorizer
feature_names = vectorizer.get_feature_names_out()

# # Interpret each prediction using eli5
for text, x, prediction in zip(data, X, predictions):
    expl = eli5.explain_prediction_sklearn(model, text, vec=vectorizer, feature_names=feature_names)
    # display('model prediction: ', prediction)
    display(eli5.show_weights(model, feature_names=feature_names))

    # display(expl)
    # Print the text, prediction, and explanation
    # display(f"Text: {text}\nPrediction: {prediction}\nExplanation:\n{eli5.format_as_text(expl)}\n")
